<a href="https://colab.research.google.com/github/xuandy05/Python-A.I./blob/master/Breakout_Game_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!pip install pyglet==1.3.2 > /dev/null 2>&1

In [0]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only

import math
import glob
import io
import base64
from IPython.display import HTML
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from IPython import display as ipythondisplay
from IPython.display import clear_output

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

In [0]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()


xdpyinfo was not found, X start can not be checked! Please install xdpyinfo!


<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1013'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1013'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [0]:
def to_grayscale(img):
    return np.mean(img, axis=2).astype(np.uint8)

def downsample(img):
    return img[::2, ::2]

def preprocess(img):
    return to_grayscale(downsample(img))

In [0]:
def transform_reward(reward):
        return np.sign(reward)

In [0]:
import keras

def fit_batch(model, gamma, start_states, actions, rewards, next_states, is_terminal):
    """Do one deep Q learning iteration.
    Params:
    - model: The DQN
    - gamma: Discount factor (should be 0.99)
    - start_states: numpy array of starting states
    - actions: numpy array of one-hot encoded actions corresponding to the start states
    - rewards: numpy array of rewards corresponding to the start states and actions
    - next_states: numpy array of the resulting states corresponding to the start states and actions
    - is_terminal: numpy boolean array of whether the resulting state is terminal
    
    """
    # First, predict the Q values of the next states. Note how we are passing ones as the mask.
    next_Q_values = model.predict([next_states, np.ones(actions.shape)])
    
    # The Q values of the terminal states is 0 by definition, so override them
    next_Q_values[is_terminal] = 0
    # The Q values of each start state is the reward + gamma * the max next state Q value
    Q_values = rewards + gamma * np.max(next_Q_values, axis=1)
    target = actions * Q_values[:, None]
    target = target[0]
    # Fit the keras model. Note how we are passing the actions as the mask and multiplying
    # the targets by the actions.
    model.fit(
        [start_states, actions], target,
        epochs=1, batch_size=len(start_states), verbose=0
    )



Using TensorFlow backend.


In [0]:

def atari_model(n_actions):
    # We assume a theano backend here, so the "channels" are first.
    ATARI_SHAPE = (105, 80, 4)

    # With the functional API we need to define the inputs.
    frames_input = keras.layers.Input(ATARI_SHAPE, name='frames')
    actions_input = keras.layers.Input((n_actions,), name='mask')

    # Assuming that the input frames are still encoded from 0 to 255. Transforming to [0, 1].
    normalized = keras.layers.Lambda(lambda x: x / 255.0)(frames_input)
    
    # "The first hidden layer convolves 16 8×8 filters with stride 4 with the input image and applies a rectifier nonlinearity."
    conv_1 = keras.layers.convolutional.Convolution2D(
        16, 8, 8, subsample=(4, 4), activation='relu'
    )(normalized)
    # "The second hidden layer convolves 32 4×4 filters with stride 2, again followed by a rectifier nonlinearity."
    conv_2 = keras.layers.convolutional.Convolution2D(
        32, 4, 4, subsample=(2, 2), activation='relu'
    )(conv_1)
    # Flattening the second convolutional layer.
    conv_flattened = keras.layers.core.Flatten()(conv_2)
    # "The final hidden layer is fully-connected and consists of 256 rectifier units."
    hidden = keras.layers.Dense(256, activation='relu')(conv_flattened)
    # "The output layer is a fully-connected linear layer with a single output for each valid action."
    output = keras.layers.Dense(n_actions)(hidden)
    # Finally, we multiply the output by the mask!
    filtered_output = keras.layers.multiply([output, actions_input])

    model = keras.models.Model(input=[frames_input, actions_input], output=filtered_output)
    optimizer = optimizer=keras.optimizers.RMSprop(lr=0.00025, rho=0.95, epsilon=0.01)
    model.compile(optimizer, loss='mse')
    return model

In [0]:
from collections import deque
# Import the gym module
import gym
env = gym.make('BreakoutDeterministic-v4')

memory = deque(maxlen = 1000000)
gamma = 0.99
alpha = 0.00025
epsilon = 1
epochs = 150
model = atari_model(4)

from keras.utils import to_categorical

for i in range(epochs):
  state = env.reset()
  state = preprocess(state)
  start_states = deque(maxlen = 4)
  actions = deque(maxlen = 4)
  next_states = deque(maxlen = 4)
  rewards = deque(maxlen = 4)
  is_terminal = deque(maxlen = 4)
  done = False
  total_reward = 0
  
  while not done:
    e = random.uniform(0,1)
    if e < epsilon or len(start_states)<4:
      action = env.action_space.sample()
    else:
      not_states = np.reshape(np.array(start_states),[1,105,80,4])
      preds = model.predict([not_states, np.ones([1,4])])
      action = np.argmax(preds)
    
    next_state, reward, done, info = env.step(action)
    next_state = preprocess(next_state)
    state = next_state
    start_states.append(state)
    actions.append(action)
    next_states.append(next_state)
    reward = transform_reward(reward)
    rewards.append(reward)
    is_terminal.append(done)
    if len(actions) == 4:
      data = np.array(actions)
      encoded = to_categorical(data, num_classes=4)
      encoded = np.amax(encoded,axis=0)
      encoded = np.reshape(encoded,[1,4])
      not_state = np.reshape(np.array(start_states),[1,105,80,4])
      not_next_state = np.reshape(np.array(next_states),[1,105,80,4])
      not_is_terminal = np.reshape(is_terminal,[1,4])
      not_rewards = np.reshape(rewards,[1,4])
      memory.append(((not_state),(encoded),np.array(not_rewards),(not_next_state),np.array(not_is_terminal)))
    total_reward += reward
  print("Epoch:", i, "Score:", total_reward)
  epsilon = min(0.1, epsilon-0.001) 
  batchsize = 500
  if len(memory) > batchsize:
    batch = random.sample(memory, batchsize)
    for start_states, actions, rewards, next_states, is_terminal in batch:
      fit_batch(model, gamma, start_states, actions, rewards, next_states, is_terminal)
env.close()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (8, 8), activation="relu", strides=(4, 4))`
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (4, 4), activation="relu", strides=(2, 2))`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("mu...)`


Epoch: 0 Score: 1.0


Epoch: 1 Score: 0.0
Epoch: 2 Score: 0.0
Epoch: 3 Score: 5.0
Epoch: 4 Score: 0.0
Epoch: 5 Score: 0.0
Epoch: 6 Score: 1.0
Epoch: 7 Score: 5.0
Epoch: 8 Score: 0.0
Epoch: 9 Score: 5.0
Epoch: 10 Score: 0.0
Epoch: 11 Score: 0.0
Epoch: 12 Score: 0.0
Epoch: 13 Score: 5.0
Epoch: 14 Score: 1.0
Epoch: 15 Score: 0.0
Epoch: 16 Score: 5.0
Epoch: 17 Score: 0.0
Epoch: 18 Score: 0.0
Epoch: 19 Score: 0.0
Epoch: 20 Score: 0.0
Epoch: 21 Score: 0.0
Epoch: 22 Score: 0.0
Epoch: 23 Score: 2.0
Epoch: 24 Score: 0.0
Epoch: 25 Score: 0.0
Epoch: 26 Score: 5.0
Epoch: 27 Score: 5.0
Epoch: 28 Score: 5.0
Epoch: 29 Score: 0.0
Epoch: 30 Score: 0.0
Epoch: 31 Score: 0.0
Epoch: 32 Score: 2.0
Epoch: 33 Score: 5.0
Epoch: 34 Score: 4.0
Epoch: 35 Score: 0.0
Epoch: 36 Score: 5.0
Epoch: 37 Score: 0.0
Epoch: 38 Score: 6.0
Epoch: 39 Score: 0.0
Epoch: 40 Score: 0.0
Epoch: 41 Score: 0.0
Epoch: 42 Score: 0.0
Epoch: 43 Score: 5.0
Epoch: 44 Score: 0.0
Epoch: 45 Score: 0.0
Epoch: 46 Score: 5.0
Epoch: 47 Score: 1.0


In [0]:
env = wrap_env(gym.make('BreakoutDeterministic-v4'))
state = env.reset()
is_done = False
states = deque(maxlen = 4)
for time_t in range(1000):
  env.render()
  if len(states)<4:
    action = env.action_space.sample()
  else:
    not_states = np.reshape(np.array(start_states),[1,105,80,4])
    preds = model.predict([not_states, np.ones([1,4])])
    action = np.argmax(preds)
  state = preprocess(state)
  states.append(state)
  state, reward, is_done, _ = env.step(action)
  if is_done:
    break

env.close()
show_video()